In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import cv2  # opencv+python == 4.9.0.80 # image processing
import matplotlib.pyplot as plt
import hdf5storage  # 0.1.19 # for loading .mat files

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf # Ensure using 2.13.0 as mentioned
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Conv2DTranspose, BatchNormalization, Activation)
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers.legacy import Adam # trying legacy Adam for better performance
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator # optional data augmentation

import seaborn as sns
from collections import Counter

In [4]:
import os
import random
import shutil

# Path to your data
base_path = os.path.expanduser("~/Desktop/uw-madison-gi-tract-image-segmentation/train")

# Function to select a random day and retain its scans for a given case
def retain_random_day(case_path):
    # Get all "day" folders within the case folder
    day_folders = [folder for folder in os.listdir(case_path) if os.path.isdir(os.path.join(case_path, folder))]
    if not day_folders:
        return
    
    # Select a random "day" folder
    random_day_folder = random.choice(day_folders)
    random_day_path = os.path.join(case_path, random_day_folder, "scans")
    
    if not os.path.exists(random_day_path):
        print(f"Scans folder not found for {random_day_folder} in {case_path}")
        return

    # Create a new directory to retain the selected scans
    output_case_path = os.path.join(case_path, "selected_scans")
    os.makedirs(output_case_path, exist_ok=True)
    
    # Move the selected scans to the new directory
    for scan_file in os.listdir(random_day_path):
        if scan_file.endswith(".png"):
            shutil.move(os.path.join(random_day_path, scan_file), os.path.join(output_case_path, scan_file))
    
    # Remove the other day folders
    for day_folder in day_folders:
        if day_folder != random_day_folder:
            shutil.rmtree(os.path.join(case_path, day_folder))
    
    # Move the selected scans back to the case folder root and delete the temporary folder
    for scan_file in os.listdir(output_case_path):
        shutil.move(os.path.join(output_case_path, scan_file), os.path.join(case_path, scan_file))
    os.rmdir(output_case_path)

# Iterate over all cases and process them
for case_folder in sorted(os.listdir(base_path)):
    case_path = os.path.join(base_path, case_folder)
    if os.path.isdir(case_path):
        print(f"Processing {case_folder}...")
        retain_random_day(case_path)

print("All cases processed.")


Processing case101...
Processing case102...
Processing case107...
Processing case108...
Processing case11...
Processing case110...
Processing case111...
Processing case113...
Processing case114...
Processing case115...
Processing case116...
Processing case117...
Processing case118...
Processing case119...
Processing case121...
Processing case122...
Processing case123...
Processing case124...
Processing case125...
Processing case129...
Processing case130...
Processing case131...
Processing case133...
Processing case134...
Processing case135...
Processing case136...
Processing case138...
Processing case139...
Processing case140...
Processing case141...
Processing case142...
Processing case143...
Processing case144...
Processing case145...
Processing case146...
Processing case147...
Processing case148...
Processing case149...
Processing case15...
Processing case154...
Processing case156...
Processing case16...
Processing case18...
Processing case19...
Processing case2...
Processing case20

In [6]:
# Function to clean up the extra "day" folder
def cleanup_day_folders(case_path):
    # List all items in the case folder
    items = os.listdir(case_path)
    
    # Identify the scan files and the extra day folder
    scan_files = [item for item in items if item.endswith(".png")]
    day_folders = [item for item in items if os.path.isdir(os.path.join(case_path, item))]
    
    if not day_folders:
        return
    
    # Move scan files from the extra day folder to the case root
    for day_folder in day_folders:
        day_folder_path = os.path.join(case_path, day_folder)
        for scan_file in os.listdir(day_folder_path):
            if scan_file.endswith(".png"):
                shutil.move(os.path.join(day_folder_path, scan_file), os.path.join(case_path, scan_file))
        # Remove all contents of the day folder
        for root, dirs, files in os.walk(day_folder_path, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        # Remove the empty day folder
        os.rmdir(day_folder_path)

# Iterate over all cases and clean up the extra "day" folders
for case_folder in sorted(os.listdir(base_path)):
    case_path = os.path.join(base_path, case_folder)
    if os.path.isdir(case_path):
        print(f"Cleaning up {case_folder}...")
        cleanup_day_folders(case_path)

print("Cleanup of all cases completed.")

Cleaning up case101...
Cleaning up case102...
Cleaning up case107...
Cleaning up case108...
Cleaning up case11...
Cleaning up case110...
Cleaning up case111...
Cleaning up case113...
Cleaning up case114...
Cleaning up case115...
Cleaning up case116...
Cleaning up case117...
Cleaning up case118...
Cleaning up case119...
Cleaning up case121...
Cleaning up case122...
Cleaning up case123...
Cleaning up case124...
Cleaning up case125...
Cleaning up case129...
Cleaning up case130...
Cleaning up case131...
Cleaning up case133...
Cleaning up case134...
Cleaning up case135...
Cleaning up case136...
Cleaning up case138...
Cleaning up case139...
Cleaning up case140...
Cleaning up case141...
Cleaning up case142...
Cleaning up case143...
Cleaning up case144...
Cleaning up case145...
Cleaning up case146...
Cleaning up case147...
Cleaning up case148...
Cleaning up case149...
Cleaning up case15...
Cleaning up case154...
Cleaning up case156...
Cleaning up case16...
Cleaning up case18...
Cleaning up cas

In [7]:
# Function to rename case folders chronologically
def rename_case_folders(base_path):
    # Get a sorted list of all case folders
    case_folders = sorted([folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))])

    # Iterate over the case folders and rename them chronologically
    for index, case_folder in enumerate(case_folders, start=1):
        old_case_path = os.path.join(base_path, case_folder)
        new_case_folder = f"Case_{index}"
        new_case_path = os.path.join(base_path, new_case_folder)
        os.rename(old_case_path, new_case_path)
        print(f"Renamed {case_folder} to {new_case_folder}")

# Call the function to rename case folders
rename_case_folders(base_path)

Renamed case101 to Case_1
Renamed case102 to Case_2
Renamed case107 to Case_3
Renamed case108 to Case_4
Renamed case11 to Case_5
Renamed case110 to Case_6
Renamed case111 to Case_7
Renamed case113 to Case_8
Renamed case114 to Case_9
Renamed case115 to Case_10
Renamed case116 to Case_11
Renamed case117 to Case_12
Renamed case118 to Case_13
Renamed case119 to Case_14
Renamed case121 to Case_15
Renamed case122 to Case_16
Renamed case123 to Case_17
Renamed case124 to Case_18
Renamed case125 to Case_19
Renamed case129 to Case_20
Renamed case130 to Case_21
Renamed case131 to Case_22
Renamed case133 to Case_23
Renamed case134 to Case_24
Renamed case135 to Case_25
Renamed case136 to Case_26
Renamed case138 to Case_27
Renamed case139 to Case_28
Renamed case140 to Case_29
Renamed case141 to Case_30
Renamed case142 to Case_31
Renamed case143 to Case_32
Renamed case144 to Case_33
Renamed case145 to Case_34
Renamed case146 to Case_35
Renamed case147 to Case_36
Renamed case148 to Case_37
Renamed cas

In [9]:
import numpy as np
from PIL import Image

base_path = os.path.expanduser("~/Desktop/uw-madison-gi-tract-image-segmentation/train")
output_path = os.path.expanduser("~/Desktop/processed_volumes")
os.makedirs(output_path, exist_ok=True)

# Function to load and stack images into a 3D array
def stack_images_to_3d(case_path):
    # Get all image files
    image_files = sorted([file for file in os.listdir(case_path) if file.endswith(".png")])
    
    # Ensure there are 144 images
    if len(image_files) != 144:
        print(f"Warning: {case_path} does not contain 144 images. It contains {len(image_files)} images.")
    
    # Load images and stack into a 3D array
    images = []
    for image_file in image_files:
        image_path = os.path.join(case_path, image_file)
        image = Image.open(image_path)
        image_array = np.array(image)
        images.append(image_array)
    
    volume = np.stack(images, axis=0)
    return volume

# Iterate over all cases and process them
for case_folder in sorted(os.listdir(base_path)):
    case_path = os.path.join(base_path, case_folder)
    if os.path.isdir(case_path):
        print(f"Processing {case_folder}...")
        volume = stack_images_to_3d(case_path)
        # Save the volume as a numpy array
        output_file = os.path.join(output_path, f"{case_folder}.npy")
        np.save(output_file, volume)

print("All cases processed and saved.")


Processing Case_1...
Processing Case_10...
Processing Case_11...
Processing Case_12...
Processing Case_13...
Processing Case_14...
Processing Case_15...
Processing Case_16...
Processing Case_17...
Processing Case_18...
Processing Case_19...
Processing Case_2...
Processing Case_20...
Processing Case_21...
Processing Case_22...
Processing Case_23...
Processing Case_24...
Processing Case_25...
Processing Case_26...
Processing Case_27...
Processing Case_28...
Processing Case_29...
Processing Case_3...
Processing Case_30...
Processing Case_31...
Processing Case_32...
Processing Case_33...
Processing Case_34...
Processing Case_35...
Processing Case_36...
Processing Case_37...
Processing Case_38...
Processing Case_39...
Processing Case_4...
Processing Case_40...
Processing Case_41...
Processing Case_42...
Processing Case_43...
Processing Case_44...
Processing Case_45...
Processing Case_46...
Processing Case_47...
Processing Case_48...
Processing Case_49...
Processing Case_5...
Processing Case

ValueError: need at least one array to stack